In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/pope/popular/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a dining table in the image?\nAnswer...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(9377), tensor(9378), tensor(9379), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(3723), tensor(3724), tensor(3725), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(7357), tensor(7358), tensor(7359), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a skis in the image?\nAnswer the que...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(13633), tensor(13634), tensor(13635), ...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a chair in the image?\nAnswer the qu...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(4593), tensor(4594), tensor(4595), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None


In [5]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [6]:
embeddings=np.concatenate(embeddings,axis=0)

In [7]:
embeddings.shape

(13624, 33, 4096)

In [8]:
gt_label=df['label'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [9]:
gt_label.shape

(13624,)

In [10]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [11]:
is_correct=[]
for i,x in enumerate(resps):
    if x.lower()=='no' and gt_label[i]==0:
        is_correct.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [12]:
is_correct.shape

(13624,)

In [13]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 13624 responses. 11850 answer correctly and 1774 answer wrongly


In [14]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 2000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:8899
validset length:2000
testset length:2725
trainset postive: 7737 negative: 1162
valset postive: 1747 negative: 253
testset postive: 2366 negative: 359


In [15]:
all_embeddings = embeddings
all_embeddings.shape

(13624, 33, 4096)

In [16]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [18]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCAKernel
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [19]:
def evaluator(n_components,i_layer,X,y):
    kpca=PCAKernel(embedding_train[:,i_layer,:],n_components,kernel='linear')
    scores=kpca.get_score2(X[:,i_layer,:])
    return auroc(scores,y)

In [20]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,3),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

found best n_components:1,layer:10 Score: 0.5910663339298764
found best n_components:1,layer:11 Score: 0.7068401845286443
found best n_components:1,layer:12 Score: 0.797405377032564
found best n_components:1,layer:13 Score: 0.8320214665004491
found best n_components:1,layer:14 Score: 0.854325088067404
found best n_components:1,layer:15 Score: 0.8571147376304042
found best n_components:1,layer:16 Score: 0.8576351102171764
found best n_components:1,layer:17 Score: 0.8611668563387037


  0%|          | 0/23 [00:00<?, ?it/s]

In [21]:
best_paras

{'n_components': 1, 'layer': 17, 'best_score': np.float64(0.8611668563387037)}

In [22]:
best_layer=best_paras['layer']
#best_layer=24

View Results on Training set and Test set

In [23]:
discriminator=PCAKernel(embedding_train[:,best_layer,:],best_paras['n_components'],kernel='linear')
scores=discriminator.get_score2(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [24]:
scores=discriminator.get_score2(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.7100797842454208 AUROC:0.8577498939423567


In [26]:
scores=discriminator.get_score2(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.6994495412844036 AUROC:0.8572064318796695


Train Linear Prob to Get Better Results

In [27]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [28]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [29]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.7009174311926606 AUROC:0.8479339387845923
